# libraries


In [9]:
     
import serial
import time
import cv2
import numpy as np

# modbus to regular


In [ ]:
import struct
from pymodbus.client import ModbusSerialClient as ModbusClient

# הגדרות החיבור
PORT = "COM11"  # שם הפורט
BAUDRATE = 230400  # בודרייט אופייני ל-MODBUS
TIMEOUT = 2  # זמן מקסימלי להמתנה לתשובה

# יצירת לקוח Modbus RTU
client = ModbusClient(
    port=PORT,
    baudrate=BAUDRATE,
    timeout=TIMEOUT,
    parity='N',
    stopbits=1,
    bytesize=8
)


def read_modbus_registers(slave_id, register_address, count=1):
    """
    קורא רגיסטרים בפרוטוקול MODBUS RTU ומחזיר את הנתונים כערכי A2D.
    """
    if not client.connect():
        print("Failed to connect to Modbus device.")
        return None

    try:
        response = client.read_input_registers(address=register_address, count=count, slave=slave_id)
        if response.isError():
            print("Error reading Modbus registers")
            return None
        return response.registers
    except Exception as e:
        print(f"Error: {e}")
        return None
    finally:
        client.close()

def convert_to_a2d(modbus_values, v_ref=5.0, resolution=16):
    """
    ממיר ערכים שהתקבלו ממודבוס לערכים אנלוגיים (A2D).
    הנחת יסוד: ערכי 16 ביט.
    """
    max_value = 2**resolution - 1  # 65535 עבור 16 ביט
    return [(val / max_value) * v_ref for val in modbus_values]

if __name__ == "__main__":
    slave_id = 1
    register_address = 0

    print("🔄 Attempting connection...")
    modbus_data = read_modbus_registers(slave_id, register_address, count=2)

    if modbus_data:
        print(f"✅ Raw Modbus Data: {modbus_data}")
        a2d_values = convert_to_a2d(modbus_data)
        print(f"📊 Converted A2D Values (V): {a2d_values}")
    else:
        print("⚠️ Failed to read data. Check connection and parameters.")

# main - GAS

In [ ]:
import serial
import time
import cv2
import numpy as np
from threading import Thread

# הגדרות תצוגה
width, height = 500, 500
font = cv2.FONT_HERSHEY_SIMPLEX
sampels_per_sec=100
time_interval=5 #sec
sampels_num=sampels_per_sec*time_interval
c3=[]
channel_values = {1: [], 2: [], 3: [], 4: []}
# לאחר חישוב ממוצעים – כאן יאוחסנו הממוצעים של כל ערוץ
last_computed_averages = None
# משתנה לשליטה שמונע הפעלת חישוב במקביל
averaging_in_progress = False

def process_channel(channel_bytes):

    swapped_bytes = channel_bytes[2:3] + channel_bytes[1:2] + channel_bytes[0:1]
    value = int.from_bytes(swapped_bytes, byteorder='big')
    return value

def process_packet(packet):

    data_bytes = packet[4:]
    channels = [data_bytes[i:i+3] for i in range(0, 12, 3)]
    return [process_channel(ch) for ch in channels]

def compute_averages_for_interval():
    """
    לאחר לחיצה על 'a', פונקציה זו תרוץ במשך 2 שניות.
    כל 0.5 שניות היא בודקת את הדגימות שנאספו באותו האינטרוול,
    לוקחת את הערך המקסימלי לכל ערוץ, ושומרת אותם ברשימה מקומית.
    בסיום 4 אינטרוולים (2 שניות), היא מחשבת ממוצע של הערכים המקסימליים לכל ערוץ
    ומעדכנת את המשתנה הגלובלי last_computed_averages.
    """
    global last_computed_averages, averaging_in_progress, channel_values

    local_max = {1: [], 2: [], 3: [], 4: []}
    intervals_collected = 0

    while intervals_collected < 4:
        time.sleep(1)  # מחכים 0.5 שניות לאיסוף דגימות
        # לכל ערוץ, מחשבים את המקסימום שהצטבר באותו האינטרוול
        for ch_num in range(1, 5):
            if channel_values[ch_num]:
                local_max[ch_num].append(max(channel_values[ch_num]))
            #else:
                #   local_max[ch_num].append(0)
        print(channel_values[3])
        # מנקים את הרשימות כדי להתחיל אינטרוול חדש
        for ch in channel_values.values():
            ch.clear()
        intervals_collected += 1

    # חישוב ממוצע של 4 הערכים המקסימליים לכל ערוץ
    averages = {}
    for ch_num in range(1, 5):
        if local_max[ch_num]:
            averages[ch_num] = sum(local_max[ch_num]) / len(local_max[ch_num])
        else:
            averages[ch_num] = 0

    last_computed_averages = averages
    averaging_in_progress = False


def compute_averages_for_interval2(c3):


    local_max=max(c3)
    return local_max




def read_serial(port='COM11', baudrate=230400):
    global last_computed_averages, averaging_in_progress
    ser = serial.Serial(port, baudrate, timeout=0.01)  # הפחתת זמן ה-Timeout לשיפור מהירות הקריאה
    print(f"Connected to {port} at {baudrate} baud.")

    try:
        while True:
            black_image = np.zeros((height, width, 3), np.uint8)
            
            # קריאת כל הנתונים הזמינים במכה אחת במקום בית אחד כל פעם
            bytes_available = ser.in_waiting
            if bytes_available >= 16:
                data = ser.read(bytes_available)
                
                # חיפוש חבילות שמתחילות ב־0xFF
                packets = [data[i:i+16] for i in range(len(data)-15) if data[i] == 0xFF]
                
                for packet in packets:
                    channels = process_packet(packet)
                    
                    # סינון לפי ערך סף
                    if channels[2] > 20000:
                        channel_values[3].append(channels[2])
                        cv2.putText(black_image, f'Channel 3: {channels[2]}', (50, 100), font, 1, (255, 0, 0), 2)  # הטקסט הקדמי
                
                # הצגת הממוצעים אם קיימים
                if last_computed_averages:
                    y_pos = 200
                    for ch_num, avg in last_computed_averages.items():
                        cv2.putText(black_image, f'Ch {ch_num} Avg Max: {avg:.2f}', (50, y_pos), font, 0.8, (0, 255, 0), 2)
                        y_pos += 40

                cv2.imshow('Channel Data', black_image)
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('a') and not averaging_in_progress:
                    averaging_in_progress = True
                    last_computed_averages = None
                    Thread(target=compute_averages_for_interval, daemon=True).start()

    except KeyboardInterrupt:
        print("Stopped by user.")
    finally:
        ser.close()
        cv2.destroyAllWindows()

# קריאה לפונקציה הראשית
read_serial(port='COM5', baudrate=230400)

Connected to COM5 at 230400 baud.


# tests


In [ ]:
import serial
import time
import cv2
import numpy as np
from threading import Thread

# הגדרות תצוגה
width, height = 500, 500
font = cv2.FONT_HERSHEY_SIMPLEX
sampels_per_sec=100
time_interval=5 #sec
sampels_num=sampels_per_sec*time_interval
c3=[]
channel_values = {1: [], 2: [], 3: [], 4: []}
# לאחר חישוב ממוצעים – כאן יאוחסנו הממוצעים של כל ערוץ
last_computed_averages = None
# משתנה לשליטה שמונע הפעלת חישוב במקביל
averaging_in_progress = False

def process_channel(channel_bytes):

    swapped_bytes = channel_bytes[2:3] + channel_bytes[1:2] + channel_bytes[0:1]
    value = int.from_bytes(swapped_bytes, byteorder='big')
    return value

def process_packet(packet):

    data_bytes = packet[4:]
    channels = [data_bytes[i:i+3] for i in range(0, 12, 3)]
    return [process_channel(ch) for ch in channels]

def compute_averages_for_interval():
    """
    לאחר לחיצה על 'a', פונקציה זו תרוץ במשך 2 שניות.
    כל 0.5 שניות היא בודקת את הדגימות שנאספו באותו האינטרוול,
    לוקחת את הערך המקסימלי לכל ערוץ, ושומרת אותם ברשימה מקומית.
    בסיום 4 אינטרוולים (2 שניות), היא מחשבת ממוצע של הערכים המקסימליים לכל ערוץ
    ומעדכנת את המשתנה הגלובלי last_computed_averages.
    """
    global last_computed_averages, averaging_in_progress, channel_values

    local_max = {1: [], 2: [], 3: [], 4: []}
    intervals_collected = 0

    while intervals_collected < 4:
        time.sleep(1)  # מחכים 0.5 שניות לאיסוף דגימות
        # לכל ערוץ, מחשבים את המקסימום שהצטבר באותו האינטרוול
        for ch_num in range(1, 5):
            if channel_values[ch_num]:
                local_max[ch_num].append(max(channel_values[ch_num]))
            #else:
                #   local_max[ch_num].append(0)
        print(channel_values[3])
        # מנקים את הרשימות כדי להתחיל אינטרוול חדש
        for ch in channel_values.values():
            ch.clear()
        intervals_collected += 1

    # חישוב ממוצע של 4 הערכים המקסימליים לכל ערוץ
    averages = {}
    for ch_num in range(1, 5):
        if local_max[ch_num]:
            averages[ch_num] = sum(local_max[ch_num]) / len(local_max[ch_num])
        else:
            averages[ch_num] = 0

    last_computed_averages = averages
    averaging_in_progress = False


def compute_averages_for_interval2(c3):


    local_max=max(c3)
    return local_max




def read_serial(port='COM5', baudrate=230400):
    global last_computed_averages, averaging_in_progress
    ser = serial.Serial(port, baudrate, timeout=0.01)  # הפחתת זמן ה-Timeout לשיפור מהירות הקריאה
    print(f"Connected to {port} at {baudrate} baud.")

    try:
        while True:
            black_image = np.zeros((height, width, 3), np.uint8)
            
            # קריאת כל הנתונים הזמינים במכה אחת במקום בית אחד כל פעם
            bytes_available = ser.in_waiting
            if bytes_available >= 16:
                data = ser.read(bytes_available)
                
                # חיפוש חבילות שמתחילות ב־0xFF
                packets = [data[i:i+16] for i in range(len(data)-15) if data[i] == 0xFF]
                
                for packet in packets:
                    channels = process_packet(packet)
                    
                    # סינון לפי ערך סף
                    if channels[2] > 1000000:
                        channel_values[3].append(channels[2])
                        cv2.putText(black_image, f'Channel 3: {channels[2]}', (50, 100), font, 1, (255, 0, 0), 2)  # הטקסט הקדמי
                
                # הצגת הממוצעים אם קיימים
                if last_computed_averages:
                    y_pos = 200
                    for ch_num, avg in last_computed_averages.items():
                        cv2.putText(black_image, f'Ch {ch_num} Avg Max: {avg:.2f}', (50, y_pos), font, 0.8, (0, 255, 0), 2)
                        y_pos += 40

                cv2.imshow('Channel Data', black_image)
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('a') and not averaging_in_progress:
                    averaging_in_progress = True
                    last_computed_averages = None
                    Thread(target=compute_averages_for_interval, daemon=True).start()

    except KeyboardInterrupt:
        print("Stopped by user.")
    finally:
        ser.close()
        cv2.destroyAllWindows()

# קריאה לפונקציה הראשית
read_serial(port='COM5', baudrate=230400)

Connected to COM5 at 230400 baud.


# main - FLAME

In [ ]:
#only print
import serial

port = 'COM16'
baudrate = 230400  

try:
    ser = serial.Serial(port, baudrate, timeout=1)
    print(f"✅ Connected to {ser.name}")
except serial.SerialException as e:
    print(f"❌ Failed to connect: {e}")
    exit()

def process_packet(packet):
    if packet.startswith(b'\xFF\x01\x09'):  # בדיקה אם זה פאקט תקין
        packet = packet[6:]  # מוחק את 6 הבתים הראשונים
        sensors = [packet[i:i+3] for i in range(0, 18, 3)]  # מחלק ל-6 רגשים (כל אחד 3 בתים)

        processed_sensors = []
        for i in range(3, 6):  # לוקח רק את רגשים 4, 5, 6
            sensor = sensors[i]
            reordered = sensor[2:3] + sensor[1:2] + sensor[0:1]  # מחליף סדר
            value = int.from_bytes(reordered, byteorder='big')  # ממיר למספר שלם
            processed_sensors.append(value)

        print(f"📊 Sensor Values (4,5,6): {processed_sensors}")  # מציג רק 3 ערכים רצויים

try:
    buffer = b""
    while True:
        if ser.in_waiting > 0:
            buffer += ser.read(24)  # קורא 24 בתים (גודל פאקט מלא)
            if len(buffer) >= 24:
                process_packet(buffer[:24])  # שולח לעיבוד
                buffer = buffer[24:]  # מנקה את החלק המעובד
except KeyboardInterrupt:
    print("\n🔌 Disconnected.")
    ser.close()


✅ Connected to COM16
📊 Sensor Values (4,5,6): [8524, 2450, 4372]
📊 Sensor Values (4,5,6): [8519, 2454, 4388]
📊 Sensor Values (4,5,6): [8519, 2449, 4370]
📊 Sensor Values (4,5,6): [8539, 2480, 4405]
📊 Sensor Values (4,5,6): [8530, 2460, 4389]
📊 Sensor Values (4,5,6): [8541, 2469, 4406]
📊 Sensor Values (4,5,6): [8542, 2470, 4396]
📊 Sensor Values (4,5,6): [8523, 2456, 4382]
📊 Sensor Values (4,5,6): [8546, 2468, 4395]
📊 Sensor Values (4,5,6): [8558, 2496, 4416]
📊 Sensor Values (4,5,6): [8538, 2474, 4406]
📊 Sensor Values (4,5,6): [8550, 2462, 4400]
📊 Sensor Values (4,5,6): [8559, 2467, 4387]
📊 Sensor Values (4,5,6): [8553, 2477, 4406]
📊 Sensor Values (4,5,6): [8555, 2474, 4417]
📊 Sensor Values (4,5,6): [8571, 2498, 4419]
📊 Sensor Values (4,5,6): [8552, 2468, 4404]
📊 Sensor Values (4,5,6): [8574, 2482, 4429]
📊 Sensor Values (4,5,6): [8556, 2458, 4399]
📊 Sensor Values (4,5,6): [8558, 2479, 4404]
📊 Sensor Values (4,5,6): [8558, 2480, 4402]
📊 Sensor Values (4,5,6): [8532, 2465, 4379]
📊 Sensor Va

In [ ]:
#image test

import serial
import cv2
import numpy as np
import threading

port = 'COM16'
baudrate = 230400  

# הגדרות תמונה
width, height = 500, 500
font = cv2.FONT_HERSHEY_SIMPLEX

# משתנה לשמירת הערכים האחרונים
latest_values = [0, 0, 0]
stop_thread = False  # משתנה לסגירת התהליך

try:
    ser = serial.Serial(port, baudrate, timeout=0.01)  # הפחתת זמן Timeout לקריאה מהירה יותר
    print(f"✅ Connected to {ser.name}")
except serial.SerialException as e:
    print(f"❌ Failed to connect: {e}")
    exit()

def process_packet(packet):
    global latest_values
    if packet.startswith(b'\xFF\x01\x09'):  # בדיקה אם זה פאקט תקין
        packet = packet[6:]  # מוחק את 6 הבתים הראשונים
        sensors = [packet[i:i+3] for i in range(0, 18, 3)]  # מחלק ל-6 רגשים (כל אחד 3 בתים)

        processed_sensors = []
        for i in range(3, 6):  # לוקח רק את רגשים 4, 5, 6
            sensor = sensors[i]
            reordered = sensor[2:3] + sensor[1:2] + sensor[0:1]  # מחליף סדר
            value = int.from_bytes(reordered, byteorder='big')  # ממיר למספר שלם
            processed_sensors.append(value)

        latest_values = processed_sensors  # מעדכן את הערכים האחרונים

# **פונקציה שמעדכנת את התצוגה במהירות גבוהה יותר**
def display_loop():
    global stop_thread
    while not stop_thread:
        # יצירת תמונה שחורה
        black_image = np.zeros((height, width, 3), np.uint8)

        # הצגת הנתונים על התמונה
        for idx, value in enumerate(latest_values):
            text = f"Sensor {idx+4}: {value}"  # רגשים 4, 5, 6
            cv2.putText(black_image, text, (50, 150 + idx * 50), font, 1, (0, 255, 0), 2)

        # הצגת התמונה
        cv2.imshow("Sensor Data", black_image)
        key = cv2.waitKey(1) & 0xFF  # מרענן את ה-GUI

        if key == ord('q'):  # אפשרות לסגור את החלון
            print("\n🔌 Disconnected.")
            stop_thread = True
            ser.close()
            cv2.destroyAllWindows()
            exit()

        # השהיה קצרה מאוד, שומר על רענון מהיר
        cv2.waitKey(1)

# **הפעלת Thread נפרד להצגת התמונה במהירות גבוהה יותר**
thread = threading.Thread(target=display_loop, daemon=True)
thread.start()

try:
    buffer = b""
    while True:
        if ser.in_waiting > 0:
            buffer += ser.read(24)  # קורא 24 בתים (גודל פאקט מלא)
            if len(buffer) >= 24:
                process_packet(buffer[:24])  # שולח לעיבוד
                buffer = buffer[24:]  # מנקה את החלק המעובד
except KeyboardInterrupt:
    print("\n🔌 Disconnected.")
    stop_thread = True
    ser.close()
    cv2.destroyAllWindows()


✅ Connected to COM16
